# Lectura y procesado de las distintas tablas

## 1. Salario Medio

In [2]:
import pandas as pd 
import numpy as np

avg_salary = pd.read_csv("datos/avg_salary_oecd_95_22.csv", usecols=["LOCATION", "MEASURE", "TIME", "Value"]) \
               .rename(columns={"LOCATION":"Country",
                                "MEASURE":"Currency",
                                "TIME":"Year",
                                "Value":"Average Salary"})

avg_salary.sort_values(['Country', 'Year'], inplace=True)

Calculamos una nueva columna que sea el crecimiento de un año a otro

In [3]:
avg_salary.sort_values(['Country', 'Year'], inplace=True)

# Calcular el cambio porcentual del PIB
avg_salary['Salary Growth'] = avg_salary.groupby('Country')['Average Salary'].pct_change() * 100

# Llenar NaN para los primeros años
avg_salary['Salary Growth'] = avg_salary.groupby('Country')['Salary Growth'].fillna(np.nan)


In [4]:
avg_salary.head()

,Country,Currency,Year,Average Salary,Salary Growth
0,AUS,USD,1995,43715.000000,NaN
1,AUS,USD,1996,44976.275594,2.885224
2,AUS,USD,1997,46355.555094,3.066682
3,AUS,USD,1998,46999.635104,1.389434
4,AUS,USD,1999,48064.870240,2.266475


## 2. Indice de Precios sobre el Consumo

In [5]:
cpi = pd.read_csv("datos/cpi_oecd_95_22.csv", usecols=["LOCATION", "TIME", "Value"]) \
        .rename(columns={"LOCATION":"Country",
                         "TIME":"Year",
                         "Value":"CPI"})

cpi.sort_values(['Country', 'Year'], inplace=True)

In [6]:
cpi.head()

,Country,Year,CPI
0,AUS,1995,4.627767
1,AUS,1996,2.615385
2,AUS,1997,0.224887
3,AUS,1998,0.860135
4,AUS,1999,1.483129


## 3. Producto Interior Bruto

In [7]:
import pandas as pd 
import numpy as np

gdp = pd.read_csv("datos/gdp_oecd_95_22.csv", usecols=["LOCATION", "TIME", "Value"]) \
        .rename(columns={"LOCATION":"Country",
                         "TIME":"Year",
                         "Value":"GDP"})

gdp.sort_values(['Country', 'Year'], inplace=True)

Calculamos una nueva columna que sea el crecimiento de un año a otro

In [8]:

gdp.sort_values(['Country', 'Year'], inplace=True)

# Calcular el cambio porcentual del PIB
gdp['GDP Growth'] = gdp.groupby('Country')['GDP'].pct_change() * 100

# Llenar NaN para los primeros años
gdp['GDP Growth'] = gdp.groupby('Country')['GDP Growth'].fillna(np.nan)


gdp.head()

,Country,Year,GDP,GDP Growth
1644,ALB,1997,8556.079,NaN
1645,ALB,1998,9451.895,10.469936
1646,ALB,1999,10792.583,14.184330
1647,ALB,2000,11927.776,10.518270
1648,ALB,2001,13159.431,10.325940


## 4. Brecha de Genero en los Salarios

In [9]:
import pandas as pd 

gender_gap = pd.read_csv("datos/gender_gap_oecd_95_22.csv", usecols=["REF_AREA", "TIME_PERIOD", "OBS_VALUE"]) \
               .rename(columns={"REF_AREA":"Country",
                                "TIME_PERIOD":"Year",
                                "OBS_VALUE":"Gender Gap"})

gender_gap.sort_values(['Country', 'Year'], inplace=True)

In [10]:
gender_gap.head()

,Country,Year,Gender Gap
20,AUS,1995,14.478114
21,AUS,1997,15.254237
22,AUS,1998,13.242009
23,AUS,1999,14.285714
24,AUS,2000,17.200000


## 5. Ratio entre Salario Mínimo y el Medio/Mediano

In [11]:
import pandas as pd 

min_to_avg_salary = pd.read_csv("datos/min_to_avg_salary_oecd_95_22.csv", usecols=["REF_AREA", "TIME_PERIOD", "AGGREGATION_OPERATION", "OBS_VALUE"]) \
                      .rename(columns={"REF_AREA":"Country",
                                       "TIME_PERIOD":"Year",
                                       "AGGREGATION_OPERATION":"AVG_MED",
                                       "OBS_VALUE":"min_to_avg_salary"})

# Dividimos la tabla en dos columnas en funcion de del atributo
min_to_avg_salary = min_to_avg_salary.pivot_table(index=['Country', 'Year'],
                                           columns='AVG_MED',
                                           values='min_to_avg_salary',
                                           aggfunc='first').reset_index()

min_to_avg_salary.columns = ['Country', 'Year', 'Ratio Minimum and Average Salaries', 'Ratio Minimum and Median Salaries']
min_to_avg_salary.sort_values(['Country', 'Year'], inplace=True)

In [12]:
min_to_avg_salary.head()

,Country,Year,Ratio Minimum and Average Salaries,Ratio Minimum and Median Salaries
0,AUS,1995,54.190476,61.402878
1,AUS,1996,52.501878,60.449827
2,AUS,1997,51.269615,59.900000
3,AUS,1998,52.150838,60.715447
4,AUS,1999,50.908399,59.110429


## 6. Población de cada pais

In [13]:
population = pd.read_csv("datos/population_oecd_95_22.csv", usecols=["REF_AREA", "TIME_PERIOD", "OBS_VALUE"]) \
               .rename(columns={"REF_AREA":"Country",
                                "TIME_PERIOD":"Year",
                                "OBS_VALUE":"Population"})

population.sort_values(['Country', 'Year'], inplace=True)

In [14]:
population.head()

,Country,Year,Population
644,AUS,1995,18004882.0
645,AUS,1996,18224767.0
646,AUS,1997,18423037.0
647,AUS,1998,18607584.0
648,AUS,1999,18812264.0


## 7. Obtención del Dataset Final

In [15]:
dataset = avg_salary.merge(gender_gap, on=['Country', 'Year'], how='outer')
dataset = dataset.merge(cpi, on=['Country', 'Year'], how='left')
dataset = dataset.merge(gdp, on=['Country', 'Year'], how='left')
dataset = dataset.merge(min_to_avg_salary, on=['Country', 'Year'], how='left')
dataset = dataset.merge(population, on=['Country', 'Year'], how='left')

dataset['GDP per capita'] = dataset['GDP'] / dataset['Population']*1000000

# Convert columns to numbers
columns_to_convert = ['Year', 'Average Salary', 'Salary Growth', 'Gender Gap', 'CPI', 'GDP', 'GDP Growth',
                      'Ratio Minimum and Average Salaries', 'Ratio Minimum and Median Salaries',
                      'Population', 'GDP per capita']

dataset[columns_to_convert] = dataset[columns_to_convert].apply(pd.to_numeric, errors='coerce')

dataset.head()


,Country,Currency,Year,Average Salary,Salary Growth,Gender Gap,CPI,GDP,GDP Growth,Ratio Minimum and Average Salaries,Ratio Minimum and Median Salaries,Population,GDP per capita
0,AUS,USD,1995,43715.000000,NaN,14.478114,4.627767,404102.992,NaN,54.190476,61.402878,18004882.0,22444.078889
1,AUS,USD,1996,44976.275594,2.885224,NaN,2.615385,424472.904,5.040772,52.501878,60.449827,18224767.0,23290.992088
2,AUS,USD,1997,46355.555094,3.066682,15.254237,0.224887,450962.701,6.240633,51.269615,59.900000,18423037.0,24478.195479
3,AUS,USD,1998,46999.635104,1.389434,13.242009,0.860135,478691.653,6.148835,52.150838,60.715447,18607584.0,25725.620962
4,AUS,USD,1999,48064.870240,2.266475,14.285714,1.483129,510947.993,6.738438,50.908399,59.110429,18812264.0,27160.366929


In [16]:
dataset.to_csv("dataset_oecd_95_22.csv", index=False)

In [17]:
dataset.info

<bound method DataFrame.info of      Country Currency  Year  Average Salary  Salary Growth  Gender Gap  \
0        AUS      USD  1995    43715.000000            NaN   14.478114   
1        AUS      USD  1996    44976.275594       2.885224         NaN   
2        AUS      USD  1997    46355.555094       3.066682   15.254237   
3        AUS      USD  1998    46999.635104       1.389434   13.242009   
4        AUS      USD  1999    48064.870240       2.266475   14.285714   
...      ...      ...   ...             ...            ...         ...   
1065     USA      USD  2020    77567.000000       5.974533   17.652495   
1066     USA      USD  2021    78894.280607       1.711141   16.864175   
1067     USA      USD  2022    77463.473025      -1.813576   16.984402   
1068     COL      NaN  2022             NaN            NaN    1.858736   
1069     CRI      NaN  2022             NaN            NaN    1.449275   

            CPI           GDP  GDP Growth  Ratio Minimum and Average Salaries  

In [18]:
nan_counts = pd.DataFrame()

for column in dataset.columns:
    if column != 'Country':
        nan_counts[column] = dataset.groupby('Country')[column].apply(lambda x: x.isna().sum())

nan_counts.reset_index(inplace=True)
nan_counts['Sum'] = nan_counts.iloc[:, 1:].sum(axis=1)
nan_counts

,Country,Currency,Year,Average Salary,Salary Growth,Gender Gap,CPI,GDP,GDP Growth,Ratio Minimum and Average Salaries,Ratio Minimum and Median Salaries,Population,GDP per capita,Sum
0,AUS,0,0,0,1,1,0,0,1,0,0,0,0,3
1,AUT,0,0,0,1,6,0,0,1,28,28,0,0,64
2,BEL,0,0,0,1,6,0,0,1,0,0,0,0,8
3,CAN,0,0,0,1,2,0,0,1,0,0,0,0,4
4,CHE,0,0,0,1,15,0,0,1,28,28,0,0,73
5,CHL,0,0,0,1,16,0,0,1,11,11,0,0,40
6,COL,1,0,1,2,3,0,0,1,3,3,0,0,14
7,CRI,1,0,1,2,15,1,0,1,15,15,0,0,51
8,CZE,0,0,0,1,3,0,0,1,0,0,0,0,5
9,DEU,0,0,0,1,1,0,0,1,20,20,0,0,43
